In [1]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
!pip install --upgrade huggingface-hub -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.4 MB/s eta 0:00

# **SQL 프롬프트**

In [ ]:
def make_prompt(ddl, question, query=''):
  prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

###SQL:
{query}"""

  return prompt

# **평가를 위한 요청 jsonl 작성 함수**

In [ ]:
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  # 경로 확인시 -> 없으면 생성
  if not Path(dir).exists():
    Path(dir).mkdir(parents=True)

  prompts = []

  # 데이터프레임 읽기
  for idx, row in df.iterrows():
    # GPT 요청 JSON 리스트 생성
    # response_format: GPT가 JSON 형식으로 응답하도록
    prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)
  # prompts 리스트를 이용하여 GPT 요청 JSON 리스트 생성
  # response_format: -> GPT 응답 데이터 형식
  # message: GPT 시스템 역할 지정 및 평가 요청 내용 포함
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]

  # 쓰기 모드로 파일 열기
  with open(Path(dir, filename), "w") as f:
    for job in jobs:
      # json.dumps -> 파이썬 객체를 JSON 문자열로 변환하는 함수
      json_string = json.dumps(job)
      f.write(json_string + "\n")

# **비동기 요청 명령**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

python api_request_parallel_processor.py \
  --requests_filepath {요청 파일 경로} \
  --save_filepath {생성할 결과 파일 경로} \
  --request_url https://api.opneai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

# **결과 jsonl 파일을 csv로 변환하는 함수**

In [ ]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
  prompts = []
  responses = []

  with open(input_file, 'r') as json_file:
    # 파일을 한줄씩 읽음
    for data in json_file:
      # json.loads(data) -> 한 줄의 JSON 데이터를 파이썬 객체(리스트 또는 딕셔너리)로 변환
      prompts.append(json.loads(data)[0]['message'][0]['content'])
      responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

  # 판다스 데이터프레임 생성
  # 컬럼명: 데이터 형식
  df = pd.DataFrame({prompt_column: prompts, response_column: responses})
  # CSV 파일로 저장
  df.to_csv(output_file, index=False)

  return df

# **기초 모델로 생성하기**

In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  # load_in_4bit=True -> 4비트 양자화 적용
  # bnb_4bit_compute_dtype=torch.float16 -> 연산을 16비트로 수행
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  # 텍스트 생성용 파이프라인을 생성
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

  return pipe

model_id = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_id)

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL.
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""
# do_sample=False -> 샘플링을 하지 않고 가장 확률이 높은 단어 선택
# do_sample=True -> 랜덤성을 추가해서 다양하게 출력
# return_full_text=False -> 입력 프롬프트는 제외하고 출력 결과만 받기
hf_pipe(example, do_sample=False, return_full_text=False, max_length=512, truncation=True)

#  SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇의 결과:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%'; (생략)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### Result:\n1\n\n### DDL:\nCREATE TABLE users (\n  user_id INT PRIMARY KEY AUTO_INCREMENT,\n  username VARCHAR(255) UNIQUE NOT NULL,\n  email VARCHAR(255) UNIQUE NOT NULL,\n  password_hash VARCHAR(255) NOT NULL,\n  date_joined DATETIME NOT NULL.\n  last_login DATETIME\n);\n\n### Question:\n사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.\n\n### SQL:\nSELECT COUNT(*) FROM users WHERE username LIKE '%admin%';\n\n### Result:\n1\n\n### DDL:\nCREATE TABLE users (\n  user_id INT PRIMARY KEY AUTO_INCREMENT,\n  username VARCHAR(255) UNIQUE NOT NULL,\n  email VARCHAR(255) UNIQUE NOT NULL,\n  password_hash VARCHAR(255) NOT NULL,\n  date_joined DATETIME NOT NULL.\n  last_login DATETIME\n);\n\n### Question:\n사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.\n\n### SQL:\nSELECT COUNT(*) FROM users WHERE username LIKE '%admin%';\n\n### Result:\n1\n\n### DDL:\nCREATE TABLE users (\n  user_id INT PRIMARY KEY AUTO_INCREMENT,\n  username"}]

# **기초 모델 성능 측정**

In [ ]:
!mkdir results

In [ ]:
from datasets import load_dataset

# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
# Dataset 객체를 Pandas DataFrame으로 변환
df = df.to_pandas()

for idx, row in df.iterrows():
  # 요청 프롬프트 생성
  prompt = make_prompt(row['context'], row['question'])
  # df.loc -> 판다스 데이터프레임에 특정 행과 열의 값을 설정하거나 가져오는 방법
  # idx는 현재 행의 인덱스
  # 'prompt'는 추가할 컬럼 이름
  df.loc[idx, 'prompt'] = prompt

# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                   return_full_text=False, max_length=512, truncation=True)

# 모델이 생성한 sql을 저장
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
# 미리 정의해둔 평가 프롬프트 생성 함수
make_requests_for_gpt_evaluation(df, eval_filepath)

In [ ]:
# GPT-4 평가 수행
!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 2500 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

In [ ]:
base_eval = change_jsonl_to_csv(f"results/{eval_filepath}", "results/y1_ko_6b_eval.csv", "prompt", "resolve_yn")
# apply -> 판다스 데이터프레임의 각 행 또는 각 열의 값에 지정한 함수를 적용할 때 사용
# lambda -> 간단한 함수 정의
# lambda 매개변수: 반환값

# lambda x: json.loads(x)['resolve_yn'] -> 각각의 데이터를 x에 저장 후
# json.loads(x)['resolve_yn'] -> json을 딕셔너리로 변환 -> resolve_yn 값 가져오기
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
# base_eval.query("resolve_yn == 'yes") ->  resolve_yn 이 yes 인 값만 필터링
# .shape[0] -> 남은 행의 개수를 반환 (정답 개수)
num_correct_answers = base_eval.query("resolve_yn == 'yes").shape[0]
num_correct_answers